In [1]:
import tkinter as Tkinter
import csv
from tkinter import filedialog as tkFileDialog
from tkinter import messagebox as tkMessageBox
from PIL import Image, ImageTk
import cv2
import numpy as np
import os
import glob
import mahotas as mt
from sklearn import svm
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
import string
import pickle
import cv2
from sklearn import svm
from scipy.cluster.vq import kmeans, vq
import numpy as np
import pandas as pd
import glob
from skimage.filters import threshold_yen
from skimage.exposure import rescale_intensity
from scipy.stats import skew
from itertools import chain
from skimage.feature import greycomatrix, greycoprops
from sklearn.preprocessing import scale
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler as sc
import sys


In [2]:
filename = ""
fr_c = 0
pressed = 0
rows = []
lab = []

In [3]:
sift = cv2.SIFT_create()
# load the model from disk
filename = 'final_model.sav'
clf_svm_rbf = pickle.load(open(filename, 'rb'))

1   
def get_all_glcm_features(gray_scale_img):
    """
    Given a grayscale image with graylevels from 0 - 255, this function returns the contrast
    and the homogeneity features of the image with the help of GLCM
    """
    # Tip: Make sure you understand the input-output of everything you write, 
    # not doing that results in bugs that make you believe the lab is long
    
    #size of co-occ matrix = number of gray levels
    image_array = np.array(gray_scale_img)
    #print('first pixel= ', image_array[0][0])
    coocurrence_matrix = greycomatrix(image_array, [1], [0])
    contrast = greycoprops(coocurrence_matrix, 'contrast')
    homogeneity = greycoprops(coocurrence_matrix, 'homogeneity')
    #mean = greycoprops(coocurrence_matrix, 'mean')
    energy = greycoprops(coocurrence_matrix, 'energy')
    #entropy = greycoprops(coocurrence_matrix, 'entropy')
    #variance = greycoprops(coocurrence_matrix, 'variance')
    correlation = greycoprops(coocurrence_matrix, 'correlation')
    return contrast, homogeneity, energy, correlation

FileNotFoundError: [Errno 2] No such file or directory: 'final_model.sav'

In [4]:
def extract_features(file):
    Test_des_list=[]
    Test_hue_arr=[]
    #Test_data='dataset\\test\\*.jpg'
    '''
    Apple_training_data='dataset\\Apples\\Test\\*.jpg'
    Orange_training_data='dataset\\Oranges\\Test\\*.jpg'
    Mango_training_data='dataset\\Mangoes\\Test\\*.jpg'
    '''
    img = file
    img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    contrast, homogeneity, energy, correlation= get_all_glcm_features((img_gray * 255).astype(np.uint8))
    kpts, des = sift.detectAndCompute(img_gray,None)
    if len(kpts) < 1:
        no_kpts = np.zeros((1, sift.descriptorSize()), np.float32)
        Test_des_list.append((file, no_kpts))
    else:
        Test_des_list.append((file, des)) 
    h,s,v=cv2.split(img)
    hue_mean=np.mean(h)  #calculate the mean of hue channel of each image
    hue_var=np.var(h) 
    flatten_h = list(np.concatenate(h).flat)
    hue_skewness=skew(flatten_h)
    image_hue_info=[]
    image_hue_info.append(hue_mean)
    image_hue_info.append(hue_var)
    image_hue_info.append(hue_skewness)
    #######################################################################
    image_hue_info.append(contrast[0][0])
    image_hue_info.append(homogeneity[0][0])
    image_hue_info.append(energy[0][0])
    image_hue_info.append(correlation[0][0])
    #########################################################################
    Test_hue_arr.append(image_hue_info) 



        # Stack all the descriptors vertically in a numpy array
    descriptors = Test_des_list[0][1]
    for file, descriptor in Test_des_list[1:]:
        descriptors = np.vstack((descriptors, descriptor))


    #kmeans works only on float, so convert integers to float
    descriptors_float = descriptors.astype(float)

    k = 60  #k means awal 60 clusters
    voc, variance = kmeans(descriptors_float, k,1)




    Test_features = np.zeros((len(Test_hue_arr), k), "float32")
    for i in range(len(Test_hue_arr)):
        words, distance = vq(Test_des_list[i][1],voc)
        for w in words:
            Test_features[i][w] += 1



    #Test_features_scaled = scale(Test_features.tolist())
    #print(Test_features[0],Test_features_scaled[0])

    #Test_hue_arr_scaled = scale(Test_hue_arr)
    #print(Test_hue_arr[0],Test_hue_arr_scaled[0])

    Test_all_features=np.concatenate((Test_hue_arr,Test_features.tolist()),axis=1).tolist()
    return(Test_all_features)

In [5]:
def removeBackground(img):

    original = img.copy()

    l = int(max(5, 6))
    u = int(min(6, 6))

    ed = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    edges = cv2.GaussianBlur(img, (3, 3), 3)
    edges = cv2.cvtColor(edges, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(edges, l, u)

    _,thresh = cv2.threshold(edges, 0, 255, cv2.THRESH_BINARY  + cv2.THRESH_OTSU)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    mask = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=0)


    data = mask.tolist()
    sys.setrecursionlimit(10**8)
    for i in  range(len(data)):
        for j in  range(len(data[i])):
            if data[i][j] !=  255:
                data[i][j] =  -1
            else:
                break
        for j in  range(len(data[i])-1, -1, -1):
            if data[i][j] !=  255:
                data[i][j] =  -1
            else:
                break
    image = np.array(data)
    image[image !=  -1] =  255
    image[image ==  -1] =  0

    mask = np.array(image, np.uint8)

    result = cv2.bitwise_and(original, original, mask=mask)
    result[mask ==  0] =  0

    img1_bgr = cv2.cvtColor(result, cv2.COLOR_RGBA2RGB)
    return img1_bgr

In [6]:
def count(img):
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        _, thresh = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY  | cv2.THRESH_TRIANGLE )

        #th2 = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,11,2)
        #th3 = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,2)
        kernal = np.ones((2, 2), np.uint8)
        #dilation = cv2.dilate(thresh, kernal, iterations=30)
        dilation = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernal, iterations=10)
        dilation = cv2.dilate(dilation, kernal, iterations=5)
        #dilation = cv2.erode(thresh, (np.ones((4, 4), np.uint8)), iterations=5)
        contours, hierarchy = cv2.findContours(dilation, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        objects = str(len(contours))
        text = int(objects)


        #cv2.imshow('Original', img)
        #cv2.imshow('Thresh', thresh)
        #cv2.imshow('Dilation', dilation)

        #cv2.waitKey(0)
        #cv2.destroyAllWindows()
        return text, thresh, dilation

In [7]:
standard_height = 300; standard_width = 300
standard_aspect_ratio = standard_width / standard_height
def zoom_adjust(img, axis):
    if axis == 0:
        zoom_ratio = standard_height / img.shape[0]
        if zoom_ratio < 1:
            img = cv2.resize(img, (int(img.shape[1]*zoom_ratio),standard_height), interpolation=cv2.INTER_AREA)
        elif zoom_ratio > 1:
            img = cv2.resize(img, (int(img.shape[1]*zoom_ratio),standard_height), interpolation=cv2.INTER_CUBIC)
    elif axis == 1:
        zoom_ratio = standard_width / img.shape[1]
        if zoom_ratio < 1:
            img = cv2.resize(img, (standard_width,int(img.shape[0]*zoom_ratio)), interpolation=cv2.INTER_AREA)
        elif zoom_ratio > 1:
            img = cv2.resize(img, (standard_width,int(img.shape[0]*zoom_ratio)), interpolation=cv2.INTER_CUBIC)
    else:
        if(standard_height < img.shape[0]):
            img = cv2.resize(img, (standard_width,standard_height), interpolation=cv2.INTER_AREA)
        else:
            img = cv2.resize(img, (standard_width,standard_height), interpolation=cv2.INTER_CUBIC)
        img = img[:,:,0:3]
    return img
def padding(img, axis):
    A = np.zeros((standard_height,standard_width,3), dtype=np.uint8)
    if axis == 0:
        bottom = int((standard_height - img.shape[0]) / 2)
        top = bottom + img.shape[0]
        A[bottom:top,:,:] = img[:,:,0:3]
    elif axis == 1:
        left = int((standard_width - img.shape[1]) / 2)
        right = left + img.shape[1]
        A[:,left:right,:] = img[:,:,0:3]
    return A
def cutting(img, axis):
    img = zoom_adjust(img, axis)
    if axis == 0:
        left = int((img.shape[1] - standard_width) / 2)
        right = left + standard_width
        img = img[:,left:right,0:3]
    elif axis == 1:
        bottom = int((img.shape[0] - standard_height) / 2)
        top = bottom + standard_height
        img = img[bottom:top,:,0:3]
    return img
def size_adjust(img, method="resize"):
    if method == "resize":
        img = zoom_adjust(img, 2)
    elif method == "padding":
        aspect_ratio = img.shape[1] / img.shape[0]
        if aspect_ratio < standard_aspect_ratio:
            img = zoom_adjust(img, 0)
            img = padding(img, 1)
        elif aspect_ratio > standard_aspect_ratio:
            img = zoom_adjust(img, 1)
            img = padding(img, 0)
        else:
            img = zoom_adjust(img, 2)
    elif method == "cutting":
        aspect_ratio = img.shape[1] / img.shape[0]
        if aspect_ratio < standard_aspect_ratio:
            img = cutting(img, 1)
        elif aspect_ratio > standard_aspect_ratio:
            img = cutting(img, 0)
        else:
            img = zoom_adjust(img, 2)
    else: print("Error: Argument 'method' not allowed")
    return img

In [8]:
class login:
   def __init__(self, master):
      self.master = master
##      self.path = "Fruit.jpg"
      master.configure(background='sky blue')      
      Tkinter.Label(root, text="FRUIT CLASSIFIER",font=("Mannberg",50),bg = 'sky blue',fg = 'red').place(x=415,y=150)
      Tkinter.Label(root, text="Username",font=(10),width=10,fg='dark green',bg = 'sky blue').place(x=525,y=300)
      Tkinter.Label(root, text="Password",font=(10),width=10,fg='dark green',bg = 'sky blue').place(x=525,y=350)
      self.e1 = Tkinter.Entry(master,width = 20,font=(10))
      self.e2 = Tkinter.Entry(master,width = 20,show='*',font=(10))
      self.e1.pack(ipady=10)
      self.e2.pack(ipady=10) 
      self.e1.place(x=675,y=300)
      self.e2.place(x=675,y=350)
      #Tkinter.Button(root, text='Quit', command=root.quit).place(x=800,y=500)
      Tkinter.Button(root, text='LOGIN', command=self.show_entry_fields,width=30,bg = 'dark green',fg = 'white').place(x=600,y=400)


   def show_entry_fields(self):
      if(self.e1.get()=="1" and self.e2.get()=="1"):
                c = Tkinter.Canvas(root,height=1200,width=1920,bg = 'sky blue')
                c.pack()

                label1 = Tkinter.Label(c, text='Selected Image', fg='white', bg='dark green',height=1,width=20,font=(1))
                label1.pack()
                c.create_window(250, 500, window=label1)

                label2 = Tkinter.Label(c, text='BG Removed Image', fg='white', bg='dark green',height=1,width=20,font=(1))
                label2.pack()
                c.create_window(600, 500, window=label2)

                label3 = Tkinter.Label(c, text='Thresh For Count', fg='white', bg='dark green',height=1,width=20,font=(1))
                label3.pack()
                c.create_window(950, 500, window=label3)

                label4 = Tkinter.Label(c, text='Dilation For Count', fg='white', bg='dark green',height=1,width=20,font=(1))
                label4.pack()
                c.create_window(1300, 500, window=label4)

                label5 = Tkinter.Label(c, text='FRUIT : ', fg='black', bg='indian red',height=2,width=20,font=(1))
                label5.pack()
                c.create_window(550, 750, window=label5)

                label6 = Tkinter.Label(c, text='FRUIT COUNT : ', fg='black', bg='indian red',height=2,width=20,font=(1))
                label6.pack()
                c.create_window(1000, 750, window=label6)

                panel1 = Tkinter.Label(root,width=300, height=300)
                panel2 = Tkinter.Label(root,width=300, height=300)
                panel3 = Tkinter.Label(root,width=300, height=300)
                panel4 = Tkinter.Label(root,width=300, height=300)
                        

                def select():
                        global filename

                        if fr_c > 0 :
                                btn2.configure(text = "SHOW RESULT")
                                label5.configure(text='FRUIT : ')
                                label6.configure(text='FRUIT COUNT : ')
                                panel1.configure(image='')
                                panel2.configure(image='')
                                panel3.configure(image='')
                                panel4.configure(image='')
                        filename = tkFileDialog.askopenfilename(filetypes = (("JPEG", "*.jpg;*.jpeg")
                                                                             ,("PNG", "*.png")
                                                                             ,("All files", "*.*") ))
                        if(filename != ""):

                                global pressed
                                pressed = 1
                                
                                #Creates a Tkinter-compatible photo image, which can be used everywhere Tkinter expects an image object.
                                img = size_adjust(cv2.imread(filename))
                                im1=Image.fromarray(cv2.cvtColor(img, cv2.COLOR_RGB2BGR))
                                img1 = ImageTk.PhotoImage(image=im1)

                                #The Label widget is a standard Tkinter widget used to display a text or image on the screen.
                                panel1.configure(image=img1)
                                panel1.image=img1
                                #The Pack geometry manager packs widgets in rows or columns.
                                panel1.pack(side = "bottom", fill = "both", expand = "yes")
                                panel1.place(x=100,y=150)                                


                        
                def result() :
                    if(pressed == 1):    
                            global fr_c
                            # convert to grayscale
                            img_f = cv2.imread(filename)
                            gray_f = cv2.cvtColor(img_f, cv2.COLOR_BGR2GRAY)

                            #remove back ground
                            img_f_bg_removed = removeBackground(img_f)
                            #----------------------------------------------------------------------------------------------
                            im2=Image.fromarray(cv2.cvtColor(size_adjust(img_f_bg_removed), cv2.COLOR_RGB2BGR))
                            imgtk2 = ImageTk.PhotoImage(image=im2)
                            #The Label widget is a standard Tkinter widget used to display a text or image on the screen.
                            panel2.configure(image=imgtk2)
                            panel2.image=imgtk2
                            #The Pack geometry manager packs widgets in rows or columns.
                            panel2.pack(side = "bottom", fill = "both", expand = "yes")
                            panel2.place(x=440,y=150)   
                            #----------------------------------------------------------------------------------------------

                            
                            try:
                                # extract haralick texture from the image
                                features1 = extract_features(img_f_bg_removed)
                                z = clf_svm_rbf.predict(features1)  
                                fr_c, img_thresh, img_dialation= count(img_f)
                                #----------------------------------------------------------------------------------------------
                                im2=Image.fromarray(img_thresh)
                                imgtk2 = ImageTk.PhotoImage(image=im2)
                                #The Label widget is a standard Tkinter widget used to display a text or image on the screen.
                                panel3.configure(image=imgtk2)
                                panel3.image=imgtk2
                                #The Pack geometry manager packs widgets in rows or columns.
                                panel3.pack(side = "bottom", fill = "both", expand = "yes")
                                panel3.place(x=780,y=150)   
                                #----------------------------------------------------------------------------------------------
                                #----------------------------------------------------------------------------------------------
                                im2=Image.fromarray(img_dialation)
                                imgtk2 = ImageTk.PhotoImage(image=im2)
                                #The Label widget is a standard Tkinter widget used to display a text or image on the screen.
                                panel4.configure(image=imgtk2)
                                panel4.image=imgtk2
                                #The Pack geometry manager packs widgets in rows or columns.
                                panel4.pack(side = "bottom", fill = "both", expand = "yes")
                                panel4.place(x=1120,y=150)   
                                #----------------------------------------------------------------------------------------------                            
                                fruit = z[0]
                                label5.configure(text='FRUIT : ' + fruit)
                                label6.configure(text='FRUIT COUNT : ' + str(fr_c))   
                            except:
                                tkMessageBox.showerror("Error",("Oops! the image u entered is too small. Please enter a bigger image"))
                                          
                            

                btn1 = Tkinter.Button(root,height=1,width=25,text="SELECT IMAGE",command=select,font=(1),fg='white',bg = 'steel blue')
                btn1.pack()
                w1 = c.create_window(750,100,window=btn1)

                btn2 = Tkinter.Button(root,height=1,width=25,text="SHOW RESULT",command=result,font=(1),fg='white',bg = 'steel blue')
                btn2.pack()
                w2 = c.create_window(750,620,window=btn2)

      else :
              # An error box
              tkMessageBox.showerror("Error","Wrong Credentials...!!\n\nPlease enter again")
            
         
root = Tkinter.Tk()
root.title("Fruit Classifier")
root.geometry("1920x1200")
login_screen=login(root)
root.mainloop( )
